In [11]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
import cv2

In [12]:
import sys, os
sys.path.append(os.path.dirname(os.path.join((os.path.pardir), "Modules")))

origin_dir = os.path.join(os.path.pardir, 'Data')


new_dir_path = os.path.join(os.path.pardir, 'Datasets')

true_parasitized_path = os.path.relpath("corrected_images/True_parasitized.csv")
false_parasitized_path = os.path.relpath("corrected_images/False_parasitized.csv")
true_uninfected_path = os.path.relpath("corrected_images/True_uninfected.csv")
false_uninfected_path = os.path.relpath("corrected_images/False_uninfected.csv")

In [13]:
# # RUN THIS CELL IF COLAB

# !pip install faiss-gpu
# !pip install dask[bag]
# !pip3 install tsnecuda==3.0.1+cu113 -f https://tsnecuda.isx.ai/tsnecuda_stable.html

# from tsnecuda import TSNE
# import dask


# # RUN THIS CELL IF COLAB

# !pip install faiss-gpu
# !pip3 install tsnecuda==3.0.1+cu113 -f https://tsnecuda.isx.ai/tsnecuda_stable.html

# from tsnecuda import TSNE

# 

# from google.colab import drive

# drive.mount('/content/gdrive')
# !git clone "https://{pat}@github.com/madhava20217/Malaria-Detection-from-Cells.git"

# !pip install -q -r "/content/Malaria-Detection-from-Cells/requirements_versionless.txt" 

# sys.path.append(os.path.dirname('/content/Malaria-Detection-from-Cells/Modules'))
# origin_dir = "/content/Data"
# new_dir_path = "/content/Datasets/"

# true_parasitized_path  = '/content/Malaria-Detection-from-Cells/EDA/corrected_images/True_parasitized.csv'
# false_parasitized_path = '/content/Malaria-Detection-from-Cells/EDA/corrected_images/False_parasitized.csv'
# true_uninfected_path   = '/content/Malaria-Detection-from-Cells/EDA/corrected_images/True_uninfected.csv'
# false_uninfected_path  = '/content/Malaria-Detection-from-Cells/EDA/corrected_images/False_uninfected.csv'


### Run this cell if you want to download data and label the datapoints

In [14]:
# from Modules.data_download import Data_Download
# from Modules.labelling import Labelling

# download = Data_Download(origin_dir)

In [15]:
parasitized_dir = os.path.join(origin_dir, 'cell_images', "Parasitized")
uninfected_dir  = os.path.join(origin_dir, 'cell_images', 'Uninfected')

true_parasitized = pd.read_csv(true_parasitized_path, index_col = 0, names = ['ImagePath'], header = 0)
false_parasitized = pd.read_csv(false_parasitized_path, index_col = 0, names = ['ImagePath'], header = 0)

true_uninfected = pd.read_csv(true_uninfected_path, index_col = 0, names = ['ImagePath'], header = 0)
false_uninfected = pd.read_csv(false_uninfected_path, index_col = 0, names = ['ImagePath'], header = 0)

In [16]:
for dataset in [true_parasitized, false_parasitized]:
    dataset['ImagePath'] = dataset['ImagePath'].apply(lambda row: os.path.join(parasitized_dir, row))

for dataset in [true_uninfected, false_uninfected]:
    dataset['ImagePath'] = dataset['ImagePath'].apply(lambda row: os.path.join(uninfected_dir, row))

In [17]:
true_parasitized['status'] = 'true_parasitized'
false_parasitized['status'] = 'false_parasitized'

true_uninfected['status'] = 'true_uninfected'
false_uninfected['status'] = 'false_uninfected'

#joining the dataframes
images = pd.concat([true_parasitized, false_parasitized, true_uninfected, false_uninfected])
images.reset_index(drop = True, inplace = True)

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
import tensorflow as tf

In [19]:
# import pandas as pd
# import numpy as np
# import tensorflow as tf
# import os

# import seaborn as sns
# import matplotlib.pyplot as plt
# import sys

# # import skimage
# # from skimage.color import rgb2hsv
# # from skimage.transform import rescale, resize
# # from tqdm import tqdm

# import sys
# import os

# from sklearn.metrics import classification_report


# import bz2, pickle, _pickle as cPickle

# import random

# # random.seed(1234)

# # module_path = os.path.abspath(os.path.join('..'))
# # if module_path not in sys.path:
# #     sys.path.append(module_path+"/Modules/Testing")
# # import testing_module

# SAVE_DIR = "../Pickled Datasets/"

# COPIES = 2
# N_DIGITS = 3
# HEIGHT = 25
# WIDTH  = 25


# PLOT_SAVE = "../Plots/loss-curves/"

# def compressed_pickle(name: str, data):
#     with bz2.BZ2File(os.path.join(SAVE_DIR, "{}.pbz2".format(name)), 'w') as f:
#         cPickle.dump(data, f)

# def decompress_pickle(file):
#     data = bz2.BZ2File(file, 'rb')
#     data = cPickle.load(data)
#     return data

In [20]:
# x_train_aug = decompress_pickle(SAVE_DIR + 'x_train_aug.pbz2')
# y_train_aug = decompress_pickle(SAVE_DIR + 'y_train_aug.pbz2')
# x_train = decompress_pickle(SAVE_DIR + 'x_train.pbz2')
# y_train = decompress_pickle(SAVE_DIR + 'y_train.pbz2')
# x_val = decompress_pickle(SAVE_DIR + 'x_val.pbz2')
# y_val = decompress_pickle(SAVE_DIR + 'y_val.pbz2')
# x_test = decompress_pickle(SAVE_DIR + 'x_test.pbz2')
# y_test = decompress_pickle(SAVE_DIR + 'y_test.pbz2')

# print("augmented: ", x_train_aug.shape, y_train_aug.shape)
# print("train: ", x_train.shape, y_train.shape)
# print("val: ", x_val.shape, y_val.shape)
# print("test: ", x_test.shape, y_test.shape)

In [21]:
# n_aug_train = x_train_aug.shape[0]
# n_train     = x_train.shape[0]
# n_val       = x_val.shape[0]
# n_test      = x_test.shape[0]

In [25]:
cnn = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), padding = 'same', activation = 'relu', input_shape = (25, 25, 1)),
    tf.keras.layers.MaxPool2D((3,3), padding = 'same'),

    tf.keras.layers.UpSampling2D((4,4)),

    tf.keras.layers.Conv2D(32, (3,3), padding = 'same', activation = 'relu'),
    tf.keras.layers.MaxPool2D((3,3), padding = 'same'),

    tf.keras.layers.Conv2D(32, (2,2), padding = 'same', activation = 'relu'),
    tf.keras.layers.MaxPool2D((2,2), padding = 'same'),


    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation = 'sigmoid')
])

cnn.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.003),
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

# history = cnn.fit(
#             x_train, y_train, batch_size = 512,
#             epochs = 50,
#             validation_data = [x_val, y_val],
#             validation_batch_size = 256,
#             callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 3),
#                          tf.keras.callbacks.ModelCheckpoint(str = 'val_loss', save_best_only = True, save_weights_only = True, filepath= "CNN_No_Aug_HSV")]
#         )


cnn.load_weights('CNN_Aug_HSV')

In [26]:
cnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 25, 25, 64)        640       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 9, 9, 64)         0         
 2D)                                                             
                                                                 
 up_sampling2d_1 (UpSampling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 36, 36, 32)        18464     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 12, 12, 32)       0         
 2D)                                                             
                                                      

In [34]:
cnn.save(filepath = "CNN_HSV_Best.tf")

INFO:tensorflow:Assets written to: CNN_HSV_Best.tf\assets


INFO:tensorflow:Assets written to: CNN_HSV_Best.tf\assets


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stop = EarlyStopping(monitor='val_loss',patience=2)

In [33]:
import tensorflow as tf

tf.

from tkinter import *
from tkinter import ttk
from tkinter import filedialog
from skimage.transform import resize
from skimage.color import rgb2hsv
from tensorflow.keras.preprocessing import image

 

class Root(Tk):
    def __init__(self,model):
        super(Root, self).__init__()
        self.title("Malaria Detection")
        self.minsize(640, 400)
        self.labelFrame = ttk.LabelFrame(self, text = "Open File")
        self.labelFrame.grid(column = 0, row = 1, padx = 20, pady = 20)
        self.button()
        self.button1() 
        self.model = model
 
    def button(self):
        self.button = ttk.Button(self.labelFrame, text = "Browse A File",command = self.fileDialog)
        self.button.grid(column = 1, row = 1)
  
    def fileDialog(self):
        self.filename = filedialog.askopenfilename(initialdir =  "/", title = "Select A File", filetype =
        (("png files","*.png"),("all files","*.*")) )
        self.label = ttk.Label(self.labelFrame, text = "")
        self.label.grid(column = 1, row = 2)
        self.label.configure(text = self.filename)
 
    def button1(self):
        self.button = ttk.Button(self.labelFrame, text = "submit", command = self.get_prediction)
        self.button.grid(column = 1, row = 20)
        
    def get_prediction(self):
        my_image = image.load_img(self.filename)  #input 
        my_image = image.img_to_array(my_image)
        my_image = rgb2hsv(resize(my_image, (25, 25)))[..., 1]
        my_image = np.expand_dims(my_image, axis=0)
        s=self.model.predict(my_image, )
        print(np.argmax(s[0]))
        if(np.argmax(s[0]) > 0.5):
            self.label.configure(text="Parasitized")
        else:
            self.label.configure(text="Uninfected") 
root = Root(cnn)
root.mainloop()

1/1 [==============================] - 0s 12ms/step
1
1/1 [==============================] - 0s 11ms/step
0


KeyboardInterrupt: 